# This file generates outputfile for debversioncompare tests using the built in version compare from dpkg.

In [115]:
from subprocess import run

def compare(version1, version2, op):
    dictionary = {"<":"lt", ">":"gt", "<=":"le", ">=":"ge", "=":"eq", "!=":"ne"}
    dpkg_op = dictionary[op] 
    command = f"dpkg --compare-versions {version1} {dpkg_op} {version2}"
    result = run(command.split())
    if result.returncode == 0:
        return 't'
    else:
        return 'f'

In [116]:
compare('1.1.1', '1.1.2', '<')

't'

In [117]:
with open("../sql/debversion_test.sql") as f:
    fin = f.readlines()

In [118]:
import re

output = open("debversion_test.out", "wt")
for line in fin:
    if line == '\n':
        continue
    output.write(line)
    if line.startswith("SELECT"):
        mask = re.compile(r"SELECT '([0-9a-zA-Z\+\-\.\:\~]+)'::debversion ([\>\<\=\!]+) '([0-9a-zA-Z\+\-\.\:\~]+)'::debversion;")
        match = mask.match(line)
        version1 = match.group(1)
        op = match.group(2)
        version2 = match.group(3)
        letter = compare(version1, version2, op)
        text = f" ?column? \n----------\n {letter}\n(1 row)\n\n"
        output.write(text)

output.close()

Comparing function using debian python implementation

In [119]:
from debian import debian_support 
def compare1(version1, version2, op):   
    vc = debian_support.version_compare(a,b)
    if vc > 0:
        print('version a > version b')
        if op in [">",">="]:
            return "t"
        else:
            return "f"
    elif vc == 0:
        print('version a == version b')
        if op  == "=":
            return "t"
        else:
            return "f"
    elif vc < 0:
        print('version a < version b')
        if op in ["<", "<="]:
            return "t"
        else:
            return "f"